# Regressão III - Tarefa

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [13]:
df = pd.read_csv('previsao_de_renda_II.csv', index_col=0)
df['Log_renda'] = np.log(df['renda'])
df.head()

,index,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda,Log_renda
data_ref,,,,,,,,,,,,,,
2015-01-01,5762,M,S,N,0,Empresário,Superior completo,Casado,Casa,47,16.717808,2.0,11138.14,9.318131
2015-01-01,1279,M,S,N,0,Assalariado,Superior incompleto,Casado,Casa,30,9.600000,2.0,2424.81,7.793508
2015-01-01,14111,M,S,N,0,Assalariado,Superior completo,Casado,Casa,28,8.208219,2.0,13749.66,9.528769
2015-01-01,8189,M,N,S,2,Assalariado,Médio,Casado,Casa,44,1.301370,4.0,2361.84,7.767196
2015-01-01,1830,F,N,S,2,Assalariado,Superior completo,Casado,Casa,33,1.254795,4.0,790.78,6.673020


## Avalie valores missing e substitua pela média

Neste exercício não vamos nos preocupar com valores *missing*. Substitua-os pela média.

In [14]:
df.isna().sum()

,0
index,0
sexo,0
posse_de_veiculo,0
posse_de_imovel,0
qtd_filhos,0
tipo_renda,0
educacao,0
estado_civil,0
tipo_residencia,0
idade,0


In [15]:
df['tempo_emprego'].fillna(value=df['tempo_emprego'].mean(), inplace=True)
df.isna().sum()

<ipython-input-15-3c33166cbaf2>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['tempo_emprego'].fillna(value=df['tempo_emprego'].mean(), inplace=True)


,0
index,0
sexo,0
posse_de_veiculo,0
posse_de_imovel,0
qtd_filhos,0
tipo_renda,0
educacao,0
estado_civil,0
tipo_residencia,0
idade,0


### Base de testes

Separe os três últimos meses como base *holdout* (base de teste).

In [16]:
df_test = df[df.index >= '2016-01-01']
df_test.size

2100000

### Primeiro modelo
 Não use *index* e não use a data. Avalie o modelo com o $R^2-ajustado$

In [17]:
res = smf.ols('Log_renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil+ tipo_residencia+ idade +tempo_emprego + qt_pessoas_residencia', data=df_test).fit()
print(f'R-quadrado: {res.rsquared:.2%}')

R-quadrado: 54.79%


### Análise de perfil

Categorize a variável ```tempo_emprego``` em 20 quantis (cada categoria deve ter aproximadamente a mesma quantidade de observações) e faça uma tabela de perfil (armazene como dataframe) para esta categorização:

- Cada linha é uma categoria
- Uma coluna para a quantidade de observações de cada categoria
- Uma coluna para a média do log-renda para cada categoria
- Uma coluna para a média do tempo de emprego para cada categoria

Faça um gráfico da média do tempo de emprego pela média do log-renda. Insira uma célula de texto e comente se te parece que alguma das transformações vistas no módulo deve fornecer bons resultados.

In [18]:
df_test['tempo_emprego_cat'], quebras = pd.qcut(df_test['tempo_emprego'], 20, duplicates='drop', retbins=True)
cat_df = df_test.groupby('tempo_emprego_cat')[['tempo_emprego', 'Log_renda']].mean()
cat_df['observações'] = df_test['tempo_emprego_cat'].value_counts().to_frame()
cat_df.columns = ['media_tempo_emprego', 'media_Log_renda', 'observações']
cat_df

<ipython-input-18-03f1b77bb2c2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['tempo_emprego_cat'], quebras = pd.qcut(df_test['tempo_emprego'], 20, duplicates='drop', retbins=True)
<ipython-input-18-03f1b77bb2c2>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cat_df = df_test.groupby('tempo_emprego_cat')[['tempo_emprego', 'Log_renda']].mean()


,media_tempo_emprego,media_Log_renda,observações
tempo_emprego_cat,,,
"(0.11699999999999999, 0.827]",0.520903,7.202604,7601
"(0.827, 1.468]",1.134215,7.257080,7464
"(1.468, 2.099]",1.790156,7.401755,7459
"(2.099, 2.912]",2.514075,7.472002,7511
"(2.912, 3.636]",3.299975,7.578685,7507
"(3.636, 4.304]",3.994033,7.660521,7545
"(4.304, 4.945]",4.624617,7.734648,7420
"(4.945, 5.759]",5.370319,7.857738,7509
"(5.759, 6.622]",6.203618,7.969381,7492


In [19]:
cat_df['y_min'] = cat_df.media_Log_renda - 2*df_test.groupby('tempo_emprego_cat')['Log_renda'].sem()
cat_df['y_max'] = cat_df.media_Log_renda + 2*df_test.groupby('tempo_emprego_cat')['Log_renda'].sem()

tab = df_test.merge(cat_df, on='tempo_emprego_cat', how='left')

tab

<ipython-input-19-6de2729eae29>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cat_df['y_min'] = cat_df.media_Log_renda - 2*df_test.groupby('tempo_emprego_cat')['Log_renda'].sem()
<ipython-input-19-6de2729eae29>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cat_df['y_max'] = cat_df.media_Log_renda + 2*df_test.groupby('tempo_emprego_cat')['Log_renda'].sem()


,index,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda,Log_renda,tempo_emprego_cat,media_tempo_emprego,media_Log_renda,observações,y_min,y_max
0,12946,F,N,S,0,Pensionista,Médio,Casado,Casa,61,7.746165,2.0,2137.61,7.667444,"(7.474, 7.746]",7.735452,7.865651,27108,7.856765,7.874537
1,8872,F,N,S,2,Servidor público,Superior completo,Casado,Casa,27,9.301370,4.0,13884.35,9.538518,"(9.101, 10.471]",9.763741,8.426507,7448,8.409078,8.443936
2,16279,F,S,S,3,Assalariado,Médio,Casado,Casa,38,10.967123,5.0,11042.47,9.309504,"(10.471, 12.474]",11.396944,8.602221,7500,8.585514,8.618928
3,8158,M,S,S,3,Servidor público,Médio,Casado,Casa,35,5.553425,5.0,1333.82,7.195802,"(4.945, 5.759]",5.370319,7.857738,7509,7.840688,7.874789
4,9356,F,S,N,0,Assalariado,Médio,Viúvo,Casa,63,9.265753,1.0,17633.03,9.777529,"(9.101, 10.471]",9.763741,8.426507,7448,8.409078,8.443936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,11611,F,N,S,1,Assalariado,Superior completo,Casado,Casa,29,8.665753,3.0,2930.40,7.982894,"(8.129, 9.101]",8.598975,8.248399,7501,8.231171,8.265626
149996,9144,F,N,S,0,Pensionista,Superior completo,Casado,Casa,65,7.746165,2.0,4084.37,8.314923,"(7.474, 7.746]",7.735452,7.865651,27108,7.856765,7.874537
149997,16583,F,N,S,0,Assalariado,Superior completo,Casado,Casa,33,10.969863,2.0,4339.66,8.375551,"(10.471, 12.474]",11.396944,8.602221,7500,8.585514,8.618928
149998,3024,F,S,S,1,Empresário,Superior completo,União,Casa,28,8.219178,3.0,9159.49,9.122546,"(8.129, 9.101]",8.598975,8.248399,7501,8.231171,8.265626


### Linearização

Experimente pelo menos duas técnicas de linearização vistas ao longo do módulo para melhorar o ajuste da variável ```tempo_emprego```. Compare o $R^2-ajustado$ e avalie se alguma delas apresentou resultados relevantes com relação ao primeiro modelo.


In [20]:
res = smf.ols('Log_renda ~ tempo_emprego',data=df_test).fit()
print(f'R-quadrado: {res.rsquared:.2%}')

R-quadrado: 52.16%


In [21]:
res = smf.ols('Log_renda ~ tempo_emprego_cat',data=df_test).fit()
print(f'R-quadrado: {res.rsquared:.2%}')

R-quadrado: 51.74%


In [22]:
res = smf.ols('Log_renda ~ I(np.log(tempo_emprego+1))',data=df_test).fit()
print(f'R-quadrado: {res.rsquared:.2%}')

R-quadrado: 40.32%


**Resposta:** Nenhuma das técnicas de linearização apresentou um resultado melhor que o modelo original

### Avaliando o modelo

Avalie os resultados dos modelos construidos na base de testes.

In [26]:
res = smf.ols('Log_renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil+ tipo_residencia+ idade + tempo_emprego + qt_pessoas_residencia', data=df_test).fit()
print('Primeiro modelo')
print(f'R-quadrado: {res.rsquared:.2%}')

Primeiro modelo
R-quadrado: 54.79%


In [27]:
res = smf.ols('Log_renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil+ tipo_residencia+ idade + tempo_emprego_cat + qt_pessoas_residencia', data=df_test).fit()
print('Modelo com a variável tempo_emprego categorizada')
print(f'R-quadrado: {res.rsquared:.2%}')

Modelo com a variável tempo_emprego categorizada
R-quadrado: 53.37%


In [28]:
res = smf.ols('Log_renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil+ tipo_residencia+ idade +I(np.log(tempo_emprego+1)) + qt_pessoas_residencia', data=df_test).fit()
print('Modelo com logaritmo da variável tempo_emprego')
print(f'R-quadrado: {res.rsquared:.2%}')

Modelo com logaritmo da variável tempo_emprego
R-quadrado: 46.38%
